# Parameter Analysis
* We would like to take the average top k runs, rather than only the top run to improve stability.
* For the next run, will they overwrite the trial_params & trial_values tables ???
* For each Trial, we should use a new db file. But this was not done for the (end-to-end / HumanPose) study, so skip the first 196 rows for the next study

In [1]:
import sqlite3
import numpy as np

In [2]:
# connect to db
conn = sqlite3.connect('safe_goal_navigation_study2.db')
cursor = conn.cursor()


In [3]:
# Execute a query to retrieve the names of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names from the query result
tables = cursor.fetchall()

# Print out the table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
studies
version_info
study_directions
study_user_attributes
study_system_attributes
trials
trial_user_attributes
trial_system_attributes
trial_params
trial_values
trial_intermediate_values
trial_heartbeats
alembic_version


In [5]:
# sample get rows
i = 0
print(f"Table name: {tables[i][0]}")
cursor.execute(f"SELECT * FROM {tables[i][0]}")
rows = cursor.fetchall()
for row in rows:
    print(row)


Table name: studies
(1, 'safe_goal_navigation2')


In [6]:
skip_rows = 0  # set to 0 for End-to-End, and 196 for HumanPose

In [8]:
# get trial_values_dict
j = 9
print(f"Table name: {tables[j][0]}")
cursor.execute(f"SELECT * FROM {tables[j][0]}")
rows = cursor.fetchall()
trial_values_dict = {row[1]:row[3] for row in rows[skip_rows:]}
print(len(trial_values_dict))

Table name: trial_values
298


In [9]:
# get trial_params_dict
print(f"Table name: {tables[8][0]}")
cursor.execute(f"SELECT * FROM {tables[8][0]}")
rows = cursor.fetchall()
trial_params_dict = {i:list(row[2:4] for row in rows if row[1]==i) for i in trial_values_dict.keys()}
print(len(trial_params_dict))

Table name: trial_params
298


### Get parameters for top K runs

In [16]:
k = 10
sorted_values = list(trial_values_dict.items())
sorted_values.sort(reverse=True, key=lambda x: x[1])
top_k_trial = sorted_values[:k]
top_trials_list = []
for trial_num, trial_value in top_k_trial:
    print(f"trial: {trial_num}, Value: {trial_value}")
    print(trial_params_dict[trial_num])
    print("")
    top_trials_list.append(trial_params_dict[trial_num])

trial: 289, Value: 194.09883135557175
[('lr_critic', 0.0005195643086454788), ('lr_actor', 0.0009145361184631777), ('hidden_size', 5.0), ('lagrangian_multiplier_init', 0.3632809429541994), ('lr_lambda', 0.07440154986219596), ('batch_size', 2.0), ('entropy_coef', 0.031220720150334714), ('max_grad_norm', 0.5926251009798884), ('cost_limit', 0.1258981046732445), ('reward_normalize', 1.0), ('cost_normalize', 1.0), ('obs_normalize', 1.0), ('kl_early_stop', 1.0), ('use_max_grad_norm', 1.0), ('use_critic_norm', 1.0), ('standardized_rew_adv', 0.0), ('standardized_cost_adv', 1.0), ('use_cost', 0.0), ('linear_lr_decay', 0.0), ('exploration_noise_anneal', 1.0), ('clip_range', 0.2789187828457436), ('update_iters', 19.0)]

trial: 299, Value: 188.56431564688683
[('lr_critic', 0.0005053017778442057), ('lr_actor', 0.0008745743727171115), ('hidden_size', 5.0), ('lagrangian_multiplier_init', 0.5870106233384039), ('lr_lambda', 0.07376333462389119), ('batch_size', 2.0), ('entropy_coef', 0.03186191143291919)

In [15]:
# avg
for i in range(len(top_trials_list[0])):
    name = top_trials_list[0][i][0]
    avg = np.mean([v[i][1] for v in top_trials_list])
    print(name,avg)

lr_critic 0.000616556762894564
lr_actor 0.0008569505055309841
hidden_size 4.4
lagrangian_multiplier_init 0.43532863033721114
lr_lambda 0.07765659316338809
batch_size 1.6
entropy_coef 0.031120492328169873
max_grad_norm 0.45074944827331764
cost_limit 0.20093043391461998
reward_normalize 1.0
cost_normalize 1.0
obs_normalize 1.0
kl_early_stop 0.8
use_max_grad_norm 0.6
use_critic_norm 1.0
standardized_rew_adv 0.0
standardized_cost_adv 1.0
use_cost 0.0
linear_lr_decay 0.0
exploration_noise_anneal 1.0
clip_range 0.27507605182512596
update_iters 18.2


In [17]:
# avg
for i in range(len(top_trials_list[0])):
    name = top_trials_list[0][i][0]
    avg = np.mean([v[i][1] for v in top_trials_list])
    print(name,avg)

lr_critic 0.0006164105624445416
lr_actor 0.000783040839475371
hidden_size 4.4
lagrangian_multiplier_init 0.4448625536594034
lr_lambda 0.07772632321402657
batch_size 1.6
entropy_coef 0.03208225424268979
max_grad_norm 0.43782069019498043
cost_limit 0.22408200753940677
reward_normalize 1.0
cost_normalize 1.0
obs_normalize 0.9
kl_early_stop 0.8
use_max_grad_norm 0.5
use_critic_norm 1.0
standardized_rew_adv 0.0
standardized_cost_adv 1.0
use_cost 0.1
linear_lr_decay 0.0
exploration_noise_anneal 1.0
clip_range 0.2764831107861129
update_iters 22.0


In [23]:
conn.close()
